<a href="https://colab.research.google.com/github/aleksejalex/special-octo-engine/blob/main/Python/01ZLMA_ex07_Binary_Data_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercise 07 of the course 01ZLMA. 

# GLM for Discrete response - Binary Data Analysis

Alternative and Binomial responses

**Bernoulli (Alternative) Model**

$$Y_{i,j} \sim Be(\pi_i) \ i = 1,\ldots,K \ \text{and} \ j = 1,\ldots, n_i.$$
$K$ is number of groups, $n_i$ is number of observations in group $i$ and $\sum_{i=1}^{K} = N$
$$ E[Y_{i,j}] = \pi_i \ \text{and} \ g(\pi_i) = \eta_i =x_i^T \beta $$


**Binomial Model**
$$Y_i = \sum_{j=1}^{n_i} Y_{i,j} \sim Bi(n_i, \pi_i)$$

**Without continuos covariate (only factor variables)**

$K$ is constant and $n_i \rightarrow \infty $

**With at least one continuos covariate**

$n_i \approx 1$ ( $n_i$ is small enough) and $K \rightarrow \infty$



## Link functions for binary data

**Logistic function:**

The logistic function is the canonical link function for binary responses, and it is CDF of the standard logistic distribution.

$$\pi_i = \frac{1}{1+e^{-x_i^T \beta}} $$ 


**Probit function:**

The CDF of the normal distribution. 
$$\pi_i = \Phi({x_i^T \beta}) $$ 


**Cauchit function:**

The CDF of the Cauchy distribution

$$\pi_i = \frac{1}{\pi}\text{arctan}(x_i^T \beta) + \frac{1}{2} $$ 


**Complementary log-log (cloglog) function:**

The inverse of the conditional log-log function (CDF of the Gumbel distribution)

$$\pi_i = 1 − e^{-e^{x_i^T \beta}}$$

The counter part of the cloglog function is log-log link function.

In [ ]:
import numpy as np
import scipy
from scipy import stats

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import datasets
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.api import abline_plot

import pandas as pd

from dfply import *  # handy module to mimic R dplyr library

from helpers import Anova

In [ ]:
x = np.tile(np.linspace(-5, 5, 101), 4).reshape(4, -1)
y = []
names =[]

for i, j in enumerate(zip(("logit", "probit", "cauchit", "cloglog"), 
                (sm.genmod.families.links.Logit(), sm.genmod.families.links.probit(), 
                 sm.genmod.families.links.cauchy(), sm.genmod.families.links.CLogLog()))):
    
    y.append(j[1].inverse(x[i]))
    names.append([j[0]]*len(x[i]))
    
y = np.array(y).flatten()
n = np.array(names).flatten()

fig, ax = plt.subplots()



sns.lineplot(x='x', y='y', data=pd.DataFrame(data={'x': x.flatten(), 'y': y, 'n': n}), ax=ax, hue='n')
    

ax.legend()
plt.show()


## Logistic regression with Titanic dataset

https://www.kaggle.com/c/titanic/data

| Variable |                 Definition                 |                       Key                      |
|:--------:|:------------------------------------------:|:----------------------------------------------:|
| survival | Survival                                   | 0 = No, 1 = Yes                                |
| pclass   | Ticket class                               | 1 = 1st, 2 = 2nd, 3 = 3rd                      |
| sex      | Sex                                        |                                                |
| Age      | Age in years                               |                                                |
| sibsp    | # of siblings / spouses aboard the Titanic |                                                |
| parch    | # of parents / children aboard the Titanic |                                                |
| ticket   | Ticket number                              |                                                |
| fare     | Passenger fare                             |                                                |
| cabin    | Cabin number                               |                                                |
| embarked | Port of Embarkation                        | C = Cherbourg, Q = Queenstown, S = Southampton |

In [ ]:
X, y = datasets.fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

titanic_train = pd.concat([X_train, y_train], axis=1)
titanic_test = pd.concat([X_test, y_test], axis=1)

In [ ]:
titanic_train

In [ ]:
titanic_train.describe(include = 'all')

In [ ]:
titanic_test.describe(include = 'all')

In [ ]:
# Number of NA's
print(titanic_train.isna().sum())
titanic_test.isna().sum()

We will modify train dataset for our academic purpose :-)

### Model where all covariates are discrete

for categorical data in pandas see https://pandas.pydata.org/docs/user_guide/categorical.html

In [ ]:
data_dis = titanic_train[['survived','pclass','sex','embarked']]

# or use dfply module
#data_dis = titanic_train >> select(titanic_train.survived, titanic_train.pclass, titanic_train.sex,
#                                   titanic_train.embarked)
print(data_dis.dtypes)

# dfply does not have mutate_if func  just mutate :(
data_dis_ = data_dis.apply(lambda x: x.astype('category') if str(data_dis['pclass'].dtype) == 'str' else x)
print(data_dis_.dtypes)

# only pandas
data_dis = data_dis[data_dis.embarked.isin(('C', 'S', 'Q'))]  # filter
data_dis = data_dis.rename(columns = {'pclass':'class_'})  # rename
data_dis = data_dis.astype('category')  # `categorize` 

# pandas pipe functionality
data_dis2=(
     data_dis
    .pipe(lambda df: df['embarked'].isin(('C', 'S', 'Q')))  # filter
    .rename({'pclass':'class_'})  # rename
    .astype('category') # `categorize` 
)

# with pandas + dfply
data_dis3 = data_dis >> \
            filter_by(data_dis.embarked.isin(('C', 'S', 'Q')) )  >> \
            transmute(survived = data_dis.survived.astype('category'),
                      class_ = data_dis.class_.astype('category'),
                      sex = data_dis.sex.astype('category'),
                      embarked = data_dis.embarked.astype('category'))
  
  

print(data_dis.describe(include='all'))
data_dis.dtypes

In [ ]:
data_dis

In [ ]:
# TODO implementation of ggpairs like function pending

g = sns.PairGrid(data=data_dis, vars=['survived', 'class_', 'sex','embarked'])
g.map_diag(sns.countplot)
g.map_lower(sns.histplot)
g.map_upper(sns.scatterplot)

In [ ]:
pd.crosstab([data_dis.embarked, data_dis.sex, data_dis.survived], [data_dis.class_], margins=True)

In [ ]:
print(pd.crosstab([data_dis.survived], [data_dis.class_], margins=True))
print(pd.crosstab([data_dis.survived], [data_dis.class_], normalize='index'))
print(pd.crosstab([data_dis.survived], [data_dis.class_], normalize='columns'))

In [ ]:
# Count observations
print(pd.crosstab([data_dis.survived], [data_dis.sex], margins=True))
# Conditional proportions given rows
print(pd.crosstab([data_dis.survived], [data_dis.sex], normalize='index'))
# Conditional proportions given columns
print(pd.crosstab([data_dis.survived], [data_dis.sex], normalize='columns'))

In [ ]:
print(pd.crosstab([data_dis.survived], [data_dis.embarked], margins=True))
print(pd.crosstab([data_dis.survived], [data_dis.embarked], normalize='columns'))


In [ ]:
table_sex = pd.crosstab([data_dis.survived], [data_dis.sex], margins=True)
table_sex.iloc[0, 0]
table_sex

In [ ]:
# Odss ratio (empirický poměr šancí)
def OR(df):
    return df.iloc[0, 0] / df.iloc[0, 1] / (df.iloc[1, 0] / df.iloc[1, 1])

table_sex = pd.crosstab([data_dis.survived], [data_dis.sex], margins=True)
print(table_sex)
print(f'Odds ratio: {OR(table_sex)}')
# Men have 

In [ ]:
#install.packages("epitools")
#library(epitools)
#oddsratio.wald(table_sex, conf.level = 0.95)

#/TODO prepare for python/

In [ ]:

c, p, dof, expected = scipy.stats.chi2_contingency(table_sex)
print(f'Statistic: {c} \n'
      f'Deg of freedom: {dof} \n'
      f'p value: {p}')

### Null model

* Compute the null model (assume that the probability of survival was the same for all passangers)

* How do we interpret estimated parameter?

## Note
endog (dependent var)for Binomial family in statsmodels can be specified in one of three ways: A 1d array of 0 or 1 values, indicating failure or success respectively. A 2d array, with two columns. The first column represents the success count and the second column represents the failure count. A 1d array of proportions, indicating the proportion of successes, with parameter var_weights containing the number of trials for each row.

In [ ]:
# dependet variable in our case `survived` must be casted to float/int as required in statsmodels documentation
# for Binomial family https://www.statsmodels.org/dev/generated/statsmodels.genmod.families.family.Binomial.html#statsmodels.genmod.families.family.Binomial


data_dis['survived'] = data_dis['survived'].astype('float')

mod0=smf.glm(formula = 'survived~1', data=data_dis,
                family=sm.families.Binomial(sm.families.links.Logit())).fit()

mod0.summary()

In [ ]:
# The chances of survival according to training data.
print(np.exp(mod0.params[0]))

# The probability of survival.
print(np.exp(mod0.params[0])/(1+np.exp(mod0.params[0])))


### Model with variable: sex

* Compute the model with one covariate sex. 

* How can we interpret estiamted coefficients? 

* Did survival depend on gender (`sex`) ?

* Perform an appropriate tests.

* Did women have a better chance of survival? 


In [ ]:
mod_sex=smf.glm(formula = 'survived~sex', data=data_dis,
                family=sm.families.Binomial(sm.families.links.Logit())).fit()

mod_sex.summary()

Use deviance to test submodels `anova(model_1,model_2,test="Chisq")`.

In [ ]:
# The chances of survival according to training data.
print(np.exp(mod_sex.params))
#sexmale:    0.08419973147842043

anova = Anova()
anova(mod_sex,mod0,test="Chisq")



In [ ]:
#Function to estimate OR with lower and upper limit of 95% CI for OR

def OR_coef(variable,model,CI):
    param = np.array(model.params)
    where = np.where(np.array(model.params.index) == variable)
    beta = param[where]
    se = np.sqrt(np.diag(model.cov_params().to_numpy()))[where]
    or_ = np.exp(beta)
    
    return pd.DataFrame(index=[variable], data={'OR': or_, 'LCL': np.exp(beta-1*scipy.stats.norm.ppf(CI/2 +0.5)*se)
                                              , 'UCL':np.exp(beta+ scipy.stats.norm.ppf(CI/2 +0.5)*se)})
OR_coef("sex[T.male]",mod_sex,0.95)

Compare with results obtained from contingency table.

### Your turn:

Estimate model with one covariate `class` and compute: 

1. Did survival depend on (`class`) ?

2. Perform an appropriate tests.

3. Compute odds ratios between classes.

4. Did passangers in second class have a better chance of survival than in third? 


In [ ]:
#1. 

In [ ]:
#2. 

In [ ]:
#3.

In [ ]:
#4.

### Model with all discrete covariates without interactions

In [ ]:
# Simple Logistic Regression model with all discrete covariates without interactions

# to work with factor in statsmodels we need to first one-hot encode variables
one_hot = pd.get_dummies(data_dis.iloc[:, data_dis.columns != 'survived'],
                         columns=list(np.array(data_dis.columns)[data_dis.columns != 'survived']),
                    drop_first=True)
one_hot = sm.add_constant(one_hot) # add intercept
mod1=sm.GLM(endog=data_dis.survived, exog=one_hot,
                family=sm.families.Binomial(sm.families.links.Logit())).fit()

mod1.summary()


Deviance tests to add/drop independent variables.

TODO /NOT IMPLEMENTED YET FOR PYTHON/ 

`drop1(model,test="Chisq")`

`add1(model,terms.to.add,test="Chisq")`

In [ ]:
#drop1(mod1,test="Chisq")  # TODO


In [ ]:
# add1(mod0,survived~sex+class+embarked, test="Chisq") TODO


In [ ]:
data_dis2 = data_dis.replace({'embarked': 'Q'}, 'C')

one_hot2 = pd.get_dummies(data_dis2.iloc[:, data_dis2.columns != 'survived'],
                         columns=list(np.array(data_dis2.columns)[data_dis2.columns != 'survived']),
                    drop_first=True)
one_hot2 = sm.add_constant(one_hot2) # add intercept

mod1=sm.GLM(endog=data_dis2.survived, exog=one_hot2,
                family=sm.families.Binomial(sm.families.links.Logit())).fit()

mod1.summary()

In [ ]:
OR_coef("sex_male",mod1,0.95)

Interpret previous result:

* By how many percentage is the chance of survival lower for  men? 

* Interpret confidence intrval and its significance.


Lets try model with second order interactions.


In [ ]:
# add1(mod1,~.^2,test="Chisq") TODO

In [ ]:

mod2_all=smf.glm(formula = 'survived~sex*embarked + sex*class_+class_*embarked', data=data_dis,
                family=sm.families.Binomial(sm.families.links.Logit())).fit()

mod2_all.summary()



In [ ]:
# step(mod2_all)  TODO

In [ ]:
mod2=smf.glm(formula = 'survived~sex*embarked + sex*class_', data=data_dis,
                family=sm.families.Binomial(sm.families.links.Logit())).fit()

mod2.summary()



In [ ]:
anova(mod2_all,mod2,test="Chisq")


Interpretation by OR in models with interactions is more complitacated, see Lecture notes.

Lets try model with merged factor levels.




In [ ]:
data_dis3 = data_dis2.replace({'class_': 2}, 1)

data_dis3.class_.unique()

In [ ]:
mod2=smf.glm(formula = 'survived~sex*embarked + sex*class_', data=data_dis3,
                family=sm.families.Binomial(sm.families.links.Logit())).fit()

mod2.summary()


In [ ]:
mod3=smf.glm(formula = 'survived~sex*embarked + sex*class_+class_*embarked', data=data_dis3,
                family=sm.families.Binomial(sm.families.links.Logit())).fit()

print(mod3.summary())



anova(mod2,mod3,test="Chisq")


## Model with continuous independent variable.


Discuss difference from models without continuous variable (again)!!!

In [ ]:
titanic_train.info()

In [ ]:
       
data_con = titanic_train >> \
                select(titanic_train.survived, titanic_train.sex, titanic_train.embarked, titanic_train.fare,
                      titanic_train.pclass, titanic_train.age)

data_con = data_con.apply(lambda x: x.astype('category') if str(x.dtype) in ('object', 'str') else x) 

print(data_con.describe(include='all'))
print(data_con.info())

    
data_con = data_con >> \
            filter_by(data_con.embarked.isin(('C', 'S', 'Q')) )  >> \
            transmute(survived = data_con.survived.astype('float'), # cast to float beacause of statsmodels requirements
                      age = data_con.age,
                      fare = data_con.fare,
                      class_ = data_con.pclass.astype('category'),
                      sex = data_con.sex.astype('category'),
                      embarked = data_con.embarked.astype('category'))

data_con = data_con.dropna(axis=0, how='any')

print(data_con.describe(include='all'))          
data_con.info()

In [ ]:
_vars=['survived', 'age','fare','class_']

fig, axes = plt.subplots(len(_vars), len(_vars), figsize=(10, 10))

sns.countplot(data=data_con, x=_vars[0], ax=axes[0, 0])
sns.boxplot(data=data_con, y=_vars[0], x=_vars[1], ax=axes[0, 1], orient='h')
sns.boxplot(data=data_con, y=_vars[0], x=_vars[2], ax=axes[0, 2] ,orient='h')
sns.countplot(data=data_con, x=_vars[3], hue=_vars[0], ax=axes[0, 3])

sns.histplot(data=data_con, y=_vars[1], ax=axes[1, 0], hue=_vars[0])
sns.histplot(data=data_con, x=_vars[1], ax=axes[1, 1], stat='frequency', kde=True)
axes[1, 2].text(0.1, 0.5, f'Corr: {round(scipy.stats.pearsonr(x=data_con[_vars[1]], y=data_con[_vars[2]])[0], 2)}')
sns.boxplot(data=data_con, x=_vars[3], y=_vars[1], ax=axes[1, 3])


sns.histplot(data=data_con, y=_vars[2], ax=axes[2, 0], hue=_vars[0])
sns.scatterplot(data=data_con, x=_vars[1], y=_vars[2], ax=axes[2, 1])
sns.histplot(data=data_con, x=_vars[2], ax=axes[2, 2],  stat='frequency', kde=True)
sns.boxplot(data=data_con, x=_vars[3], y=_vars[2], ax=axes[2, 3])


sns.histplot(data=data_con, x=_vars[3], y=_vars[0], ax=axes[3, 0])
sns.histplot(data=data_con, x=_vars[1], hue=_vars[3], ax=axes[3, 1])
sns.histplot(data=data_con, x=_vars[2], hue=_vars[3], ax=axes[3, 2])
sns.countplot(data=data_con, x=_vars[3], ax=axes[3, 3])


for i, k in enumerate(_vars):
    for j, l in enumerate(_vars):
    
            
        if i < 3:
            axes[i, j].tick_params(
                        axis='x',          # changes apply to the x-axis
                        which='both',      # both major and minor ticks are affected
                        bottom=False,      # ticks along the bottom edge are off
                        top=False,         # ticks along the top edge are off
                        labelbottom=False)
            axes[i, j].set_xlabel('')
        if j > 0:
            axes[i, j].set_ylabel('')
            axes[i, j].tick_params(
                        axis='y',         
                        which='both',      
                        left=False,      
                        top=False,         
                        labelleft=False)
        if j == 0:               
            axes[i, j].set_ylabel(k)
        if i == 3 :               
            axes[i, j].set_xlabel(l)

plt.show()

# TODO It needs define own functionality based on PairGrid which will distinguish categorical vs categorical
# categorical vs quantitative and quantitaive vs quantitative variables

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(x='sex', y='age', hue='survived', ax = ax, data=data_con, showmeans=True)
ax.set_xlabel('Gender')
ax.set_ylabel('Age')
ax.set_title('Gender boxplot')

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(x='class_', y='fare', hue='survived', ax = ax, data=data_con, showmeans=True)
ax.set_xlabel('Class')
ax.set_ylabel('Fare')
ax.set_title('Class x Fare')

Continuous variable as factor

In [ ]:
data_con_fac = data_con >> \
  mutate(age = pd.cut(data_con.age,(-np.inf, 15, 50, np.inf), labels=["child","adult","senior"]))
data_con_fac

In [ ]:
g = sns.PairGrid(data=data_con_fac, vars=list(data_con_fac.columns))  # TODO needs custom implementation

g.map_lower(sns.histplot)
g.map_diag(sns.histplot)
g.map_upper(sns.scatterplot)


In [ ]:

mod_0= smf.glm(formula = 'survived~1', data=data_con_fac,
                family=sm.families.Binomial(sm.families.links.Logit())).fit()

print(mod_0.summary())


In [ ]:

mod_age_fac = smf.glm(formula = 'survived~age', data=data_con_fac,
                family=sm.families.Binomial(sm.families.links.Logit())).fit()

print(mod_age_fac.summary())

np.exp(mod_age_fac.params)

Is the chance decreasing with increasing age?

In [ ]:
anova(mod_age_fac,mod_0,test="Chisq")



In [ ]:

mod_age = smf.glm(formula = 'survived~np.divide(age, 10)', data=data_con,
                family=sm.families.Binomial(sm.families.links.Logit())).fit()
print(mod_age.summary())
np.exp(mod_age.params)

Question:

* With increasing age by 10 years, chance to survive decreased by 11%. 

* What do you think about causality in this result?

In [ ]:
anova(mod_age,mod_0,test="Chisq")


Question:

* Can we compare by deviance test models `mod_age` and `mod_age_fac`?
* Which model do you prefere and why?
* For which approach (factorized or continuous) saturated model is useful and why?


In [ ]:

#mod_sat_fac = smf.glm(formula = 'survived~sex*age*embarked*class_', data=data_con >> \
#                          mutate(age=data_con.age.astype('category'),
#                                 fare=data_con.fare.astype('category')),
#                family=sm.families.Binomial(sm.families.links.Logit())).fit()
#mod_sat_fac.summary()

Your turn:

Consider a model with continuos variables `age`, `fare`, and any factor variable. 

* Create factor `child`, which takes values 1 (child) and 0 (adult).
* Create factor from varaible `fare`, where each level break is by 10 pounds.
* Estimate a model, where the chance of survival depends on factorized `fare` and `sex` and `child`.
* What percentage is the chance of survival lower for adult compare to child? 
* Depends the probability of survival on fare? Test it.
* Assume that the chance of survival increases with exponential increasig fare. How the chance of survival increased if the person spent an extra 10 pound for a ticket? 
* Build a model where the probabilty of survival depends on both `age` and `fare`. Are both covariates significant?
* 

Next Exercises (8 and 9):

* Logistic regression and binary classification (ROC, accuracy, ...)
* Residual analysis
* Prediction and confidence intervals
* Logistic regression and ML approach

